In [1]:
import yaml
from create_network import *
from jinja2 import Environment, FileSystemLoader
from model_merging.task_vectors import MTLTaskVector
from model_merging.utils import aggregate_task_vectors
from utils import torch_load

In [22]:
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('config/mtl.yaml.j2')
rendered_yaml = template.render()
mm_config = yaml.safe_load(rendered_yaml)

model_classes = {
  "split": MTLDeepLabv3,
  "mtan": MTANDeepLabv3
}

In [ ]:
# torch_save(model, 'logging/pt_model.pt')
# torch_load('logging/model_test.pt')

In [3]:
pt_model = torch_load(mm_config["model_merging"]["pt_model_file"])
task_vectors = [MTLTaskVector(pt_model, ft_file) for ft_file in mm_config["model_merging"]["ft_model_files"]]

In [6]:
mtl_task_vector = aggregate_task_vectors(task_vectors, mm_config)

Norm of multi-task task vector:  tensor(22514.1562)


In [169]:
train_tasks_str = ' + '.join(task.title() for task in mtl_task_vector.tasks.keys())
print(f"Dataset: {mm_config['model_merging']['dataset'].title()} | Training Task: {train_tasks_str}")

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pt_model = model_classes[mm_config["model_merging"]["network"]](mtl_task_vector.tasks).to(device)
ft_models = {
    next(iter(ft_model.tasks.keys())): ft_model
    for ft_file in mm_config["model_merging"]["ft_model_files"]
    for ft_model in [torch_load(ft_file)]
}

Dataset: Nyuv2 | Training Task: Seg + Depth


In [ ]:
# i want to get the decoders (which are task specific) from the ft models and put them in the pt model
pt_model_dict = pt_model.state_dict()
for i, task in enumerate(pt_model.tasks):
    print(f"Task {i}: {a}")
    # ft_model = ft_models[task]
    # 1 get the dict
    # 2 get the dict of only the task specific
    # 3 update the keys of the dict
    # 4 update the pt model dict
